In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
df = pd.read_csv("african_crises.csv")

In [3]:
df.head(5)

,case,cc3,country,year,systemic_crisis,exch_usd,domestic_debt_in_default,sovereign_external_debt_default,gdp_weighted_default,inflation_annual_cpi,independence,currency_crises,inflation_crises,banking_crisis
0,1,DZA,Algeria,1870,1,0.052264,0,0,0.0,3.441456,0,0,0,crisis
1,1,DZA,Algeria,1871,0,0.052798,0,0,0.0,14.149140,0,0,0,no_crisis
2,1,DZA,Algeria,1872,0,0.052274,0,0,0.0,-3.718593,0,0,0,no_crisis
3,1,DZA,Algeria,1873,0,NaN,0,0,0.0,11.203897,0,0,0,no_crisis
4,1,DZA,Algeria,1874,0,0.051308,0,0,0.0,-3.848561,0,0,0,no_crisis


In [4]:
df.describe()

,case,year,systemic_crisis,exch_usd,domestic_debt_in_default,sovereign_external_debt_default,gdp_weighted_default,inflation_annual_cpi,independence,currency_crises,inflation_crises
count,1059.000000,1059.000000,1059.000000,964.000000,1059.000000,1059.000000,1059.000000,968.000000,1059.000000,1059.000000,1059.000000
mean,35.613787,1967.767705,0.077432,42.685441,0.039660,0.152975,0.006402,90.963404,0.776204,0.132200,0.129367
std,23.692402,33.530632,0.267401,113.174239,0.195251,0.360133,0.043572,2136.787695,0.416984,0.349847,0.335765
min,1.000000,1860.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-28.502137,0.000000,0.000000,0.000000
25%,15.000000,1951.000000,0.000000,0.078697,0.000000,0.000000,0.000000,2.087577,1.000000,0.000000,0.000000
50%,38.000000,1973.000000,0.000000,0.852350,0.000000,0.000000,0.000000,5.770044,1.000000,0.000000,0.000000
75%,56.000000,1994.000000,0.000000,8.149925,0.000000,0.000000,0.000000,11.588750,1.000000,0.000000,0.000000
max,70.000000,2014.000000,1.000000,744.306139,1.000000,1.000000,0.400000,66279.892370,1.000000,2.000000,1.000000


In [5]:
def df_info(dataframe):
    display(
        pd.concat(
            [
                dataframe.dtypes.rename("dtypes"), 
                dataframe.nunique().rename("n_unique"), 
                dataframe.isna().sum().rename("nan_vals")
            ], 
            axis=1
        )
    )

In [6]:
df_info(df)

,dtypes,n_unique,nan_vals
case,int64,13,0
cc3,object,13,0
country,object,13,0
year,int64,155,0
systemic_crisis,int64,2,0
exch_usd,float64,702,95
domestic_debt_in_default,int64,2,0
sovereign_external_debt_default,int64,2,0
gdp_weighted_default,float64,6,0
inflation_annual_cpi,float64,936,91


In [7]:
for column in df.select_dtypes(exclude=[int, float]):
    print(column)
    print(df[column].unique())
    print("-"*100)

cc3
['DZA' 'AGO' 'CAF' 'CIV' 'EGY' 'KEN' 'MUS' 'MAR' 'NGA' 'ZAF' 'TUN' 'ZMB'
 'ZWE']
----------------------------------------------------------------------------------------------------
country
['Algeria' 'Angola' 'Central African Republic' 'Ivory Coast' 'Egypt'
 'Kenya' 'Mauritius' 'Morocco' 'Nigeria' 'South Africa' 'Tunisia' 'Zambia'
 'Zimbabwe']
----------------------------------------------------------------------------------------------------
banking_crisis
['crisis' 'no_crisis']
----------------------------------------------------------------------------------------------------


### Transforming binary object value into boolean

In [8]:
df["banking_crisis"] = np.where(df["banking_crisis"] == "crisis", 1, 0)

## Converting countries to geo locations

In [9]:
url = "https://gist.githubusercontent.com/tadast/8827699/raw/f5cac3d42d16b78348610fc4ec301e9234f82821/countries_codes_and_coordinates.csv"

In [10]:
geo_df = pd.read_csv(url, skipinitialspace=True)

In [11]:
geo_df.head(5)

,Country,Alpha-2 code,Alpha-3 code,Numeric code,Latitude (average),Longitude (average)
0,Afghanistan,AF,AFG,4,33.0000,65.0
1,Albania,AL,ALB,8,41.0000,20.0
2,Algeria,DZ,DZA,12,28.0000,3.0
3,American Samoa,AS,ASM,16,-14.3333,-170.0
4,Andorra,AD,AND,20,42.5000,1.6


In [12]:
df = pd.merge(
    df, geo_df[["Latitude (average)", "Longitude (average)"]],
    left_on="country", right_on=geo_df["Country"], 
    how="left"
)

In [13]:
df = df.drop(columns=["cc3", "country"])

`case` serves as Unique ID, this is not required in our case and we won't use it either

In [14]:
df = df.drop(columns=["case"])

# Impute missing features

Since imputation, normalization and standardization procedures depend on all of the feature values we first need to split our dataset in train and test.
We will do this using `sklearn.model_selection.train_test_split` functionality:

In [15]:
from sklearn.model_selection import train_test_split

test_size = 0.1

df_train, df_test = train_test_split(df, test_size=test_size, random_state= 42)

In [16]:
df_info(df_train)

,dtypes,n_unique,nan_vals
year,int64,149,0
systemic_crisis,int64,2,0
exch_usd,float64,639,83
domestic_debt_in_default,int64,2,0
sovereign_external_debt_default,int64,2,0
gdp_weighted_default,float64,6,0
inflation_annual_cpi,float64,853,74
independence,int64,2,0
currency_crises,int64,3,0
inflation_crises,int64,2,0


As we can see, there are 116 missing values in our train set, 100 `annual_inflation_cpi` values and 16 `exch_usd` values. Let's focus on this features:

In [17]:
sort_columns = ["Latitude (average)", "Longitude (average)", "year"]
df_train = df_train.sort_values(by=sort_columns).interpolate()

In [18]:
df_train[['inflation_annual_cpi','exch_usd']].describe()

,inflation_annual_cpi,exch_usd
count,953.000000,953.000000
mean,93.744599,42.359815
std,2153.693317,110.332617
min,-28.502137,0.000000
25%,2.119180,0.195500
50%,5.717000,0.868100
75%,11.765000,8.456600
max,66279.892370,744.306139


In [19]:
df_train[['inflation_annual_cpi','exch_usd']].isna().sum()/df_train.shape[0]

inflation_annual_cpi    0.0
exch_usd                0.0
dtype: float64

In [20]:
df_test[['inflation_annual_cpi','exch_usd']].isna().sum()/df_train.shape[0]

inflation_annual_cpi    0.017838
exch_usd                0.012592
dtype: float64

About 10.5% of `inflation_annual_cpi` and 1.6% of `exch_usd` are missing. We can keep both columns by imputing missing with mean values:

In [21]:
groupby_columns = ["Latitude (average)", "Longitude (average)"]
ann_inf_cpi_mean = (
    df_train
    .groupby(groupby_columns, as_index=False)
    [["inflation_annual_cpi", "exch_usd"]]
    .mean()
)

In [22]:
df_test.set_index(["Latitude (average)", "Longitude (average)"], inplace=True)
ann_inf_cpi_mean.set_index(["Latitude (average)", "Longitude (average)"], inplace=True)

In [23]:
df_test = df_test.combine_first(ann_inf_cpi_mean).reset_index()

In [24]:
df_info(df_test)

,dtypes,n_unique,nan_vals
Latitude (average),float64,13,0
Longitude (average),float64,10,0
banking_crisis,int64,2,0
currency_crises,int64,2,0
domestic_debt_in_default,int64,2,0
exch_usd,float64,94,0
gdp_weighted_default,float64,3,0
independence,int64,2,0
inflation_annual_cpi,float64,100,0
inflation_crises,int64,2,0


# Normalization & Standardization